## Criação do Dataset de Guias de Onda com Distribuição de perdas por decil.
## Como é um dataset para sser utilizado com o Pytorch, é necessário criar uma classe que implemmenta a classe Dataset, que é abstrata no Pytorch.
## Além disso, é importante implementar o método ToTensor(), alterando a disposição das dimensões da imagem para utilização dos modelos adiante.

In [ ]:
class WaveGuideDataset(Dataset):
    """Light Waveguide dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.deciles_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.deciles_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.deciles_frame.iloc[idx, 0])
        image = io.imread(img_name)
        deciles = self.deciles_frame.iloc[idx, 2:12]
        deciles = np.array([deciles]).astype('float').reshape(-1, 10)
        sample = {'image': image, 'deciles': deciles}

        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, deciles = sample['image'], sample['deciles']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'deciles': torch.from_numpy(deciles)}